# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This series of notebook demoes the use of ABBA with python.

If you managed to create an environment with PyImageJ and DeepSlice, you will be able, by running this notebook, to perform a partially automated registration of demo mouse brain sections to the a BrainGLobe atlas.

For this notebook to run, an atlas will need to be downloaded, as well as some sample dataset.

The dataset is downloaded from https://zenodo.org/record/6592478


**THIS DOES NOT 'REALLY' WORK, I DO NOT HAVE A DATASET TO TEST WITH ONE OF THE BRAINGLOBE ATLASES.**

Please share if you have one or if you know a nicely available public one.

In [ ]:
# core dependencies
import os
import time
from pathlib import Path

from bg_atlasapi import show_atlases
from bg_atlasapi import utils

from abba_python.Abba import Abba

In [ ]:
# Demo dataset for automated slices registration
zenodo_demo_slices_url = 'https://zenodo.org/record/6592478/files/'


# Only one section every five section is used for this demo
demo_sections = [
    'S00.tif',
    'S05.tif',
    'S10.tif',
    'S15.tif',
    'S20.tif',
    'S25.tif',
    'S30.tif',
    'S35.tif',
    'S40.tif',
    'S45.tif',
    'S50.tif',
    'S55.tif',
    'S60.tif',
    'S65.tif',
    'S70.tif',
    'S75.tif',
    'S80.tif']


def download_if_necessary(base_path, section_name):
    output_path = Path(base_path + section_name)
    if not output_path.exists():
        utils.check_internet_connection()
        url = zenodo_demo_slices_url + section_name + '?download=1'
        utils.retrieve_over_http(url, output_path)


def download_test_images(base_path):
    [download_if_necessary(base_path, section) for section in demo_sections]



## 1. Download test sections if necessary

    

In [ ]:

basePath = os.getcwd() + '/images/'
download_test_images(basePath)


## 2. Choose a BrainGlobe Atlas

In [ ]:
show_atlases()

## 3. ABBA initialization

In [ ]:

# -- FOR DEBUGGING
# import imagej.doctor
# imagej.doctor.checkup()
# imagej.doctor.debug_to_stderr()

abba = Abba('azba_zfish_4um') # Simply put the name of the BrainGlobe atlas
abba.show_bdv_ui()  # creates and show a bdv view
    
# !! Warning : it takes time... first : downloading the atlas if not present
# it can take up to a minute...

## 4. Import sections into ABBA

In [ ]:
# import sections into ABBA
files = [basePath + section for section in demo_sections]
abba.import_from_files(filepaths=files)

# ALL REGISTRATIONS AND COMMANDS BELOW ARE PERFORMED ON THE SELECTED SLICES!!
# since we want to register all of them, we select all of them
abba.select_all_slices()

In [ ]:
# we want to avoid saturation in the display. This does not matter for
# all registration methods EXCEPT for DeepSlice, which takes in rgb images
abba.change_display_settings(0, 0, 500)
abba.change_display_settings(1, 0, 1200)

# programmatic way to show (or hide) sections and channels
abba.get_bdv_view().setSelectedSlicesVisibility(True)
abba.get_bdv_view().setSelectedSlicesVisibility(0, True)

# This data does not make a lot of sense... mouse sections with fish...

## 5. Position sections along the atlas axis

In [ ]:
# Good luck with that, this can be done programmatically with a bit of pain now. 
# However in practice in absence of DeepSlice, you need to deal with that manually:
# cf https://www.youtube.com/watch?v=sERGONVw4zE

## 5. Elastix affine registration

In [ ]:
# a round of elastix registration, affine
# the channel 0 of the dataset (DAPI) is registered with the Nissl Channel of the atlas (0)
# and the channel 1 of the dataset (mainly autofluo) is registered with the autofluo channel of the atlas (1)
# these two channels have equal weights in the registration process
abba.register_slices_elastix_affine(channels_slice_csv='1',
                                    channels_atlas_csv='0',
                                    pixel_size_micrometer=40)

## 6. Elastix spline registration

In [ ]:
# optional: a round of elastix registration, spline
# same channels as in the affine registration
# 5 control points along x = very coarse spline (and thus maybe unnecessary)
# abba.register_elastix_spline(
#    nb_control_points=5,
#    atlas_image_channels=[0, 1],
#    slice_image_channels=[0, 1],
#    pixel_size_micrometer=40).get()

# a round of elastix registration, affine
# same channels as in the affine registration 
# 16 control points = reasonable spline, which allows for local corrections, without deforming two much the section
abba.register_slices_elastix_spline(channels_slice_csv='1',
                                    channels_atlas_csv='0',
                                    nb_control_points_x=16,
                                    pixel_size_micrometer=20)

## 7. Wait for end of all registrations

In [ ]:
# all tasks/registrations are enqueued and executed asynchronously
# if you need to wait before saving, then wait for all tasks to be finished:
abba.wait_for_end_of_tasks()

## 8. Saving the result

In [ ]:
save_dir = os.path.join(os.getcwd(), 'temp', 'notebook1', 'state')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


abba.state_save(save_dir+"/state.json") # full absolute path needed

In [ ]:
# EXTRA : manipulation of slices programmatically

# slices selection and manipulation
abba.mp.selectSlice(abba.mp.getSlices().get(2)) # select the last slice
abba.mp.getReslicedAtlas().setRotateY(0.05) # Small correction in Y slicing of the atlas
abba.mp.deselectSlice(abba.mp.getSlices()) # deselect all

# The slices are always sorted from small z to high z. To keep track of who's who, reference them before moving them
slice30 = abba.mp.getSlices().get(0) 
slice40 = abba.mp.getSlices().get(1)
slice50 = abba.mp.getSlices().get(2)

# move slices along the slicing axis
abba.mp.moveSlice(slice50,9.5)
abba.mp.moveSlice(slice40,8.2)
abba.mp.moveSlice(slice30,7.5)

# For a registration : let's select all slices
abba.mp.selectSlice(abba.mp.getSlices()) # select all